In [1]:
import os 
from pathlib import Path
abspath = Path('C:\\Users\\harkum\\Documents\\6311\\Software')
os.chdir(abspath)

In [2]:
import pandas as pd 

# Read the IVM6311 regmap data from page 1 and page 2 
data = pd.read_excel('raw_data/IVM6311_rev0_regmap.xlsx', sheet_name='PAG0')
data['Page'] = [0 for _ in range(data.shape[0])]
data.head()
data1 = pd.read_excel('raw_data/IVM6311_rev0_regmap.xlsx', sheet_name='PAG1 - IFIR COEFF')
data1['Page'] = [1 for _ in range(data1.shape[0])]

# concatinate both pages data
data = pd.concat([data, data1])
data.head()


,Hex,Dec,MSB,LSB,Default,Bit7,Bit6,Bit5,Bit4,Bit3,...,Register Name,Attribute,Bin - Default,User,Clocking,Reset,Block,Sync Reset,Sync Write,Page
0,STATUS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,00,0.0,7.0,0.0,0E,(reserved),(reserved),(reserved),(reserved),fro_sel_0p8125mhz,...,System CTRL,000NNNNN,1110.0,0000YYYY,SMB,C,RFU,NaN,NaN,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Frequency selection of ckg_froclk_muxed, it's ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,01,1.0,15.0,8.0,00,(reserved),(reserved),(reserved),(reserved),(spare),...,Software reset,0000NNNP,0.0,0000000Y,SMB,S,RFU,NaN,NaN,0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [3]:
data['Register Name'] = data['Register Name'].apply( lambda x : x.replace(' ','_').strip() if isinstance(x, str) else 0 )

In [21]:
# Seperate the Register names, Hex and Page number 
pd.DataFrame(data.groupby(['Page','Hex', 'Register Name']).groups.keys(), columns=['Page','Hex','Register Name']).to_csv('Register_names.csv')


In [61]:
data[data['Register Name'] ==  'System_CTRL']

,Hex,Dec,MSB,LSB,Default,Bit7,Bit6,Bit5,Bit4,Bit3,...,Register Name,Attribute,Bin - Default,User,Clocking,Reset,Block,Sync Reset,Sync Write,Page
1,00,0.0,7.0,0.0,0E,(reserved),(reserved),(reserved),(reserved),fro_sel_0p8125mhz,...,System_CTRL,000NNNNN,1110.0,0000YYYY,SMB,C,RFU,NaN,NaN,0


In [62]:
def create_register_class(register_data:str):
    with open('dut/IVM6311.py', 'w') as file:
        file.write(
            register_data
        )
# for each register create the python file inside the dut IVM6311
data['Register Name'].apply( lambda x : create_register_class(x) if isinstance(x, str) else x)



register_data = ' '.join([
f'''
            
class {RegisterName}:

    def __init__(self) -> None:
        self.address = 0x{data['Hex'].iloc[index]}
    
    @property
    def x(self):
        return self._value
        
    @x.setter
    def x(self,value):
        self._value = value 
        
            '''
for index,RegisterName in enumerate(data['Register Name'] )
if isinstance(RegisterName, str)  
])
create_register_class(register_data=register_data)

In [63]:
from dut.IVM6311 import PAD_settings

obj = PAD_settings()
obj.x = 10
obj.x

10

In [64]:
def string_formating(RegisterName,Bit1):
    return  f'''
class {RegisterName}:

    def __init__(self) -> None:
        pass 
        
    @property
    def {Bit1}(self):
        return self._value
        
    @{Bit1}.setter
    def {Bit1}(self,value):
        self._value = value 
'''

In [65]:
import neattext as nt 

In [66]:
def bit_string(Bit_name, Bit_number,Bit_description=''):
    return f'''
    @property
    def {Bit_name}(self):
        """
{Bit_description}
        """
        self._{Bit_name}_number = {Bit_number}
        return 'dummy Value instead real read vlaue'
        
    @{Bit_name}.setter
    def {Bit_name}(self,value):
        # before modify the existing value , read the register value 
        Reg_value =  self.{Bit_name}
        lsb = self._{Bit_name}_number # access the lsb it 
        value = value + Reg_value # we can call bit number here itself 
        print(f'write operation with set value ')
         
    '''

##### Register file writing 

In [67]:
import re 
reg_string = []


for index,RegisterName in enumerate(data['Register Name']):
    if isinstance(RegisterName,str):
        string = f'''
class {RegisterName}:

    def __init__(self) -> None:
        self.address = 0x{data['Hex'][index]} 
        '''
        
        # catch all the bits 
        for bit_number in range(8):
            Bit = data[f'Bit{bit_number}'].iloc[index]
            if isinstance(Bit, str) : 
                if  Bit != '(reserved)' : 
                    if Bit != '(spare)' :
                        # bitmask = 0
                        # if re.search('\[(.*?)\]''',string=Bit):
                        #     bitmask = re.findall('\[(.*?)\]''',string=Bit)[0]
                        #     bitmask = bitmask.split(':')
                        #     print(bitmask)
                        #     bitmask = 2**(int(bitmask[0])+int(bitmask[1])+1)-1
                        #     print(f'{Bit} : {bitmask}')
                        Bit = re.sub('\[(.*?)\]''','',string=Bit).strip()
                        if not (Bit_description := data[f'Bit{bit_number}'].iloc[index+1]):
                            Bit_description = ''
                        string = string+bit_string(Bit_name=Bit,Bit_number=bit_number,Bit_description=Bit_description)

        reg_string.append(string)


create_register_class(' '.join(reg_string))
reg_string
# data['Bit1'].iloc[17]

KeyError: 524

In [ ]:
# registers = [ register_name for _,register_name in data['Register Name'].value_counts().to_dict().values() if register_name]
# unique_registers = [register_name for register_name in data["Register Name"].unique() if register_name  ]
# numeber_of_registers = len(registers)
# print(f'number of the Registers {numeber_of_registers}')
# print(f'number of unique registers {len(unique_registers)}')
# data['Register Name'].value_counts().to_dict().keys()



In [ ]:
import re 
string = 'seq_step_time_pu[1:0]'

re.findall('\[(.*?)\]',string=string)
re.sub('\[(.*?)\]''','',string=string)

'seq_step_time_pu'

In [ ]:
from dut.IVM6311 import PAD_settings
class PAD_settings:

    def __init__(self) -> None:
        self.address = 0xFC 
        
    @property
    def tdm_pad_di_en(self):
        """
TDM PADs input enable.
0 = disabled
1 = enabled
        """
        self._tdm_pad_di_en_number = 2
        return 'dummy Value instead real read vlaue'
        
    @tdm_pad_di_en.setter
    def tdm_pad_di_en(self,value):
        # before modify the existing value , read the register value 
        Reg_value =  self.tdm_pad_di_en
        lsb = self._tdm_pad_di_en_number # access the lsb it 
        value = value + Reg_value # we can call bit number here itself 
        print(f'write operation with set value ')
obj = PAD_settings()
obj.tdm_pad_di_en

'dummy Value instead real read vlaue'

#### Parse the Register Field Description

In [ ]:
data.columns.to_list()[0]

'Hex'

In [ ]:
for i in range(0,8):
    data[f'Bit{i}']=data[f'Bit{i}'].apply( lambda x : x if isinstance(x,str) else 0)

In [ ]:
for index, RegisterName in enumerate(data['Register Name']):
    if RegisterName != 0 :
        if Bit_name := data['Bit0'].iloc[index]  :
            if (Bit_description := data['Bit0'].iloc[index+1]):
                print(Bit_description) 

Power Up
0 = Power-down
1 = Normal operation
Software hard-reset.
Interrupt clear mode
0 = clear on read
1 = clear on write (default)
Clock monitor fault
Clock missing fault mask
0 = INTN is not affected (default)
1 = INTN is affected
Over-voltage from BST
Over-voltage from BST mask
0 = INTN is not affected (default)
1 = INTN is affected
Clock monitor status
0 = clock error
1 = clock OK

Bst bso ovp status
0 = ovp not present
1 = ovp present

Boot (otp loading, platform detection and i2c_load) status
0 = boot not completed
1 = boot completed

Enable of tdm interface. This field is dynamically gated according to the scenario.
0 = disabled
1 = enabled (default)
Enable of shutdown sequence according to a bst ovp.
0 = interrupt only (default)
1 = interrupt and shutdown
In internal feedback noise-gate, selects the mod clocks status. It's meaningful only if REG14[3] = 0
0 = clk off in ng
1 = clk on in ng (dafault)
Signal path enable
0 = Signal path disabled
1 = Signal path enabled
Setting of

In [77]:
data.groupby(['Page','Register Name']).groups.items()

dict_items([((0, 0), Index([  0,   2,   4,   6,   8,  10,  12,  14,  15,  16,
       ...
       510, 511, 512, 513, 514, 516, 517, 518, 520, 522],
      dtype='int64', length=431)), ((0, 'ANA_SPARE_Register'), Index([67], dtype='int64')), ((0, 'ANA_gain_Register'), Index([269], dtype='int64')), ((0, 'AUTO_mode_setting'), Index([56], dtype='int64')), ((0, 'BST_Comp_Register_1'), Index([360], dtype='int64')), ((0, 'BST_Comp_Register_2'), Index([362], dtype='int64')), ((0, 'BST_Comp_Register_3'), Index([364], dtype='int64')), ((0, 'BST_Comp_Register_4'), Index([366], dtype='int64')), ((0, 'BST_Comp_Register_5'), Index([368], dtype='int64')), ((0, 'BST_Register_1'), Index([393], dtype='int64')), ((0, 'BST_Register_10'), Index([411], dtype='int64')), ((0, 'BST_Register_11'), Index([413], dtype='int64')), ((0, 'BST_Register_12'), Index([415], dtype='int64')), ((0, 'BST_Register_13'), Index([417], dtype='int64')), ((0, 'BST_Register_14'), Index([419], dtype='int64')), ((0, 'BST_Register_15'),

: 

In [ ]:
import json 
with open('regmap_IVM6311.json', 'r') as file :
    regmap = json.load(file)
regmap

{'RegMap': [{'PAGE': 0,
   'REG': '00',
   'POS': 3,
   'Name': 'fro_sel_0p8125mhz',
   'Length': 1,
   'Option': "Frequency selection of ckg_froclk_muxed, it's meaningfull only if REG00[1]=1. \t00 = 3.25MHz\t01 = 1.625MHz\t1X = 0.8125MHz",
   'Attribute': 'N',
   'Match': ''},
  {'PAGE': 0,
   'REG': '00',
   'POS': 2,
   'Name': 'fro_sel_1p625mhz',
   'Length': 1,
   'Option': 'None',
   'Attribute': 'N',
   'Match': ''},
  {'PAGE': 0,
   'REG': '00',
   'POS': 1,
   'Name': 'fro_sel',
   'Length': 1,
   'Option': 'Frequency selection of ckg_froclk_muxed. \tThe effective selection is = boot_done & REG00[1] &  OTP_FRO_SEL\t0 = 13MHz\t1 = divided version according to REG00[3:2]',
   'Attribute': 'N',
   'Match': ''},
  {'PAGE': 0,
   'REG': '00',
   'POS': 0,
   'Name': 'powerup',
   'Length': 1,
   'Option': 'Power Up\t0 = Power-down\t1 = Normal operation',
   'Attribute': 'N',
   'Match': ''},
  {'PAGE': 0,
   'REG': '01',
   'POS': 0,
   'Name': 'hard_reset',
   'Length': 1,
   'Opt

In [ ]:
import pandas as pd 
df_data = pd.DataFrame(data.get('RegMap'))
df_data.head()

""


In [ ]:
df_data.head()

""
